In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

agg = pd.read_csv('../raw_data/aggregate_tool_info.csv')
agg.head()

,indexing,seed_type,pairwise_alignment,count
0,BWT-FM,NNN,HD,1
1,BWT-FM,NNN,SW,1
2,BWT-FM,NNN,Semi-Global,1
3,BWT-FM,NNY,NW,2
4,BWT-FM,NNY,SW,2


In [7]:
agg = agg.rename(columns={"indexing":"source","seed_type":"type","pairwise_alignment":"target","count":"value"})
agg = agg.dropna()
agg

,source,type,target,value
0,BWT-FM,NNN,HD,1
1,BWT-FM,NNN,SW,1
2,BWT-FM,NNN,Semi-Global,1
3,BWT-FM,NNY,NW,2
4,BWT-FM,NNY,SW,2
5,BWT-FM,NNY,SW & NW,2
6,BWT-FM,YNN,Banded Myers Bit Vector,1
7,BWT-FM,YNN,HD,4
8,BWT-FM,YNN,Non-DP Heuristic,4
9,BWT-FM,YNN,SW,4


In [8]:
from ipysankeywidget import SankeyWidget
from floweaver import *

size = dict(width=1200, height=600)

nodes = {
    'index': ProcessGroup(['hashing', 'BWT-FM', 'BWT-FM + hashing', 'BWT-ST', 'suffix array', 'suffix tree']),
    'pairwise': ProcessGroup(['Banded Myers Bit Vector', 'HD', 'NW', 'SW',
                             'SW & NW', 'SW & Semi-Global', 'Non-DP Heuristic',
                             'Landau-Vishkin', 'Rabin-Karp Algorithm',
                             'Sparse DP', 'Semi-Global', 'Myers Bit-Vector']),
}

#nodes = {
#    'index': ProcessGroup(['Hashing', 'BWT-FM', 'BWT-ST', 'Suffix Array', 'Suffix Tree']),
#    'pairwise': ProcessGroup(['Banded Myers Bit Vector', 'Based on the Wrapper, Local and Global', 'Global', 'Local',
#                             'Hamming Distance', 'Local & Global', 'Local & Semi-Global', 'Non-DP Heuristic',
#                             'Hamming, Local, and Global Alignment', 'Landau-Vishkin', 'Rabin-Karp Algorithm',
#                             'Sparse DP', 'Semi-Global']),
#}

ordering = [
    ['index'],  
    ['pairwise'],
]

bundles = [
    Bundle('index', 'pairwise'),
]

sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, agg).to_widget(**size, margins=dict(top=0, bottom=0, left=50, right=100))

SankeyWidget(layout=Layout(height='600', width='1200'), links=[{'source': 'index^*', 'target': 'pairwise^*', '…

In [9]:
index_part = Partition.Simple('process', [
    'BWT-FM',
    ('BWT-FM & Hashing', ['BWT-FM + hashing']),
    ('Hashing', ['hashing']),
    ('Other Suffix', ['BWT-ST', 'suffix array', 'suffix tree']),
])

pairwise_part = Partition.Simple('process', [
    ('Smith-Waterman', ['SW']),
    ('Hamming Distance', ['HD']),
    ('Needleman-Wunsch', ['NW']),
    ('Other DP', ['Rabin-Karp Algorithm', 'Landau-Vishkin', 'Sparse DP', 'Banded Myers Bit Vector', 
                  'Semi-Global', 'Myers Bit-Vector']),
    'Non-DP Heuristic',
    ('Multiple Methods', ['SW & NW', 'SW & Semi-Global',])
])

#pairwise_part = Partition.Simple('process', [
#    'Local',
#    'Hamming Distance',
#    'Global',
#    'Non-DP Heuristic',
#    ('Multiple Methods', ['Based on the Wrapper, Local and Global', 'Local & Global', 'Local & Semi-Global', 
#                          'Hamming, Local, and Global Alignment']),
#    ('Other DP', ['Rabin-Karp Algorithm', 'Landau-Vishkin', 'Sparse DP', 
#                          'Banded Myers Bit Vector', 'Semi-Global'])
#])

nodes['index'].partition = index_part
nodes['pairwise'].partition = pairwise_part
weave(sdd, agg).to_widget(**size, margins=dict(top=0, bottom=0, left=140, right=160))

SankeyWidget(groups=[{'id': 'index', 'type': 'process', 'title': '', 'nodes': ['index^BWT-FM', 'index^BWT-FM &…

In [10]:
part_by_fixed = Partition.Simple('type', [
    ('Yes', ['YNN', 'YNY', 'YYN', 'YYY']),
    ('No', ['NNN', 'NNY', 'NYN', 'NYY']),
])
part_by_spaced = Partition.Simple('type', [
    ('Yes', ['NYN', 'NYY', 'YYN', 'YYY']),
    ('No', ['NNN', 'NNY', 'YNN', 'YNY'])
])
part_by_chained = Partition.Simple('type', [
    ('Yes', ['NNY', 'NYY', 'YNY', 'YYY']),
    ('No', ['NNN', 'NYN', 'YNN', 'YYN'])
])
part_by_index = Partition.Simple('source', [
    'BWT-FM', 
    ('BWT-FM & Hashing', ['BWT-FM + hashing']),
    ('Hashing', ['hashing']),
    ('Other Suffix', ['BWT-ST', 'suffix array', 'suffix tree']),
])
pal = ['darkorange', 'mediumorchid', 'cornflowerblue', 'navy']
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=part_by_index)
weave(sdd, agg, palette=pal).to_widget(**size, margins=dict(top=0, bottom=0, left=140, right=160)).auto_save_png(
    '../figures/Figure3.png')

SankeyWidget(groups=[{'id': 'index', 'type': 'process', 'title': '', 'nodes': ['index^BWT-FM', 'index^BWT-FM &…

In [11]:
nodes['fixed'] = Waypoint(part_by_fixed)
#nodes['chained'] = Waypoint(part_by_chained)

ordering = [
    ['index'],
    ['fixed'],
    ['pairwise']
]

bundles = [
    Bundle('index', 'pairwise', waypoints=['fixed']),
]
pal = ['darkorange', 'cornflowerblue']
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=part_by_fixed)

weave(sdd, agg, palette=pal).to_widget(**size, margins=dict(top=0, bottom=0, left=150, right=150)).auto_save_png(
    '../figures/Figure3_a.png')

SankeyWidget(groups=[{'id': 'index', 'type': 'process', 'title': '', 'nodes': ['index^BWT-FM', 'index^BWT-FM &…

In [12]:
#nodes['fixed'] = Waypoint(part_by_fixed)
nodes['spaced'] = Waypoint(part_by_spaced)
#nodes['chained'] = Waypoint(part_by_chained)
ordering = [
    ['index'],
    ['spaced'],
    ['pairwise']
]

bundles = [
    Bundle('index', 'pairwise', waypoints=['spaced']),
]

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=part_by_spaced)

weave(sdd, agg, palette=pal).to_widget(**size, margins=dict(top=0, bottom=0, left=150, right=150)).auto_save_png(
    '../figures/Figure3_b.png')

SankeyWidget(groups=[{'id': 'index', 'type': 'process', 'title': '', 'nodes': ['index^BWT-FM', 'index^BWT-FM &…

In [13]:
#nodes['fixed'] = Waypoint(part_by_fixed)
#nodes['spaced'] = Waypoint(part_by_spaced)
nodes['chained'] = Waypoint(part_by_chained)
ordering = [
    ['index'],
    ['chained'],
    ['pairwise']
]

bundles = [
    Bundle('index', 'pairwise', waypoints=['chained']),
]

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=part_by_chained)

weave(sdd, agg, palette=pal).to_widget(**size, margins=dict(top=0, bottom=0, left=150, right=150)).auto_save_png(
    '../figures/Figure3_c.png')

SankeyWidget(groups=[{'id': 'index', 'type': 'process', 'title': '', 'nodes': ['index^BWT-FM', 'index^BWT-FM &…